In [7]:
# page
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name='NEWS_LIST', con=con, index=False,
                            if_exists='append')
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')
        
def db_delete():
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [8]:
def db_select():
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con=con)
        except Exception as e:
            print(str(e))
        return NEWS_LIST

In [9]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/selection/p'):
        if p.text:
            body.append(re.sub(punc, '', p.text))
    full_body = ' '.join(body)
    
    return full_body

In [10]:
page = 58
max_page = 0
REG_DATE = '20200819'

In [11]:
while True:
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                           .format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({'URL':[url], 'TITLE':[a.text], 'ARTICLE':[article]})
        df_list.append(df)
        
    if df_list:
        df_10 = pd.concat(df_list)
        db_save(df_10)
        
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num=int(a.text)
            if max_page < num:
                max_page = num
        except Exception as e:
            pass
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')
    
    if (len(span) <= 0) & (page > max_page):
        break
    else:
        page = page + 1
        
    time.sleep(1)

15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
13 건 저장완료..


In [12]:
print(db_select())

                                        URL  \
0    https://v.daum.net/v/20200819084702255   
1    https://v.daum.net/v/20200819084617234   
2    https://v.daum.net/v/20200819084529215   
3    https://v.daum.net/v/20200819084517208   
4    https://v.daum.net/v/20200819084251134   
..                                      ...   
242  https://v.daum.net/v/20200819030053787   
243  https://v.daum.net/v/20200819021530691   
244  https://v.daum.net/v/20200819010010384   
245  https://v.daum.net/v/20200819004221292   
246  https://v.daum.net/v/20200819000017909   

                                                 TITLE ARTICLE  
0                         에기평, 코로나19 확산 방지 투명칸막이 무상 제공          
1                     중국 선전시, 전지역 커버하는 '5G SA' 네트워크 구축          
2                    상반기 전세계 모바일게임 지출 42조 돌파..반기 기준 최고          
3                5분 충전에 600km 주행 가능..전기車 리튬금속전지 개발 본격화          
4                        배달의민족, 비 피해 소상공인에 10만원 포인트 지원          
..                                           